# Qubit operations

## 1-D qubits

Система, яка має два базисних стани (наприклад, 0 та 1) і може знаходитись у стані суперпозиції, називається кубітом (квантовим бітом)

У лабораторній роботі буде використовуватися бібліотека Cirq

In [1]:
import cirq


Simple 1-D qubits are defined like this:

In [2]:
qubit = cirq.GridQubit(0, 0)

To be able to use a 1-D qubit in Cirq one has to define a Circuit

In [4]:
circuit = cirq.Circuit.from_ops(cirq.X(qubit) ** 0.5,
                                cirq.measure(qubit, key='m'))
print(circuit)

(0, 0): ───X^0.5───M('m')───


Now let's create a simulator and run the circuit

In [6]:
simulator = cirq.Simulator()
oneQubitResult = simulator.run(circuit, repetitions=50)
print(oneQubitResult)

m=11111000111111100100100010000111110111000111010011


## Multi dimensional qubits

In [7]:
dimensions = 3

qubits = [cirq.GridQubit(i, j) for i in range(dimensions) for j in range(dimensions)]

print(qubits)

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]


Qubit placing

In [11]:
line = cirq.google.line_on_device(cirq.google.Bristlecone, length=20)
print(line)

(0, 5)━━(0, 6)
        ┃
        (1, 6)━━(1, 7)
                ┃
        (2, 6)  (2, 7)━━(2, 8)
        ┃               ┃
        (3, 6)━━(3, 7)  (3, 8)━━(3, 9)
                ┃               ┃
                (4, 7)━━(4, 8)  (4, 9)━━(4, 10)
                        ┃               ┃
                        (5, 8)━━(5, 9)  (5, 10)
                                ┃       ┃
                                (6, 9)━━(6, 10)


Now we are setting up a circuit and printing it

In [13]:
n = 20
depth = 2

circuit = cirq.Circuit.from_ops(
    cirq.SWAP(cirq.LineQubit(j), cirq.LineQubit(j + 1))
    for i in range(depth)
    for j in range(i % 2, n - 1, 2)
)
circuit.append(cirq.measure(*cirq.LineQubit.range(n), key='all'))
print(circuit)

0: ────×───────M('all')───
       │       │
1: ────×───×───M──────────
           │   │
2: ────×───×───M──────────
       │       │
3: ────×───×───M──────────
           │   │
4: ────×───×───M──────────
       │       │
5: ────×───×───M──────────
           │   │
6: ────×───×───M──────────
       │       │
7: ────×───×───M──────────
           │   │
8: ────×───×───M──────────
       │       │
9: ────×───×───M──────────
           │   │
10: ───×───×───M──────────
       │       │
11: ───×───×───M──────────
           │   │
12: ───×───×───M──────────
       │       │
13: ───×───×───M──────────
           │   │
14: ───×───×───M──────────
       │       │
15: ───×───×───M──────────
           │   │
16: ───×───×───M──────────
       │       │
17: ───×───×───M──────────
           │   │
18: ───×───×───M──────────
       │       │
19: ───×───────M──────────


Now we will optimize the circuit to XMON and print it

In [14]:
xmonOptimized = cirq.google.optimized_for_xmon(
    circuit=circuit, 
    new_device=cirq.google.Bristlecone, 
    qubit_map=lambda qb: line[qb.x])

print(xmonOptimized)

                                 ┌──────────┐   ┌───────┐   ┌────────────┐            ┌───────────┐                     ┌──────────┐   ┌────────────┐   ┌───────┐            ┌───────┐            ┌───────┐   ┌───────┐   ┌──────┐   ┌──────┐   ┌──────┐
(0, 5): ────Y^-0.5───@───Y^0.5────@──────────────Y^-0.5──────────────────────@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────M('all')───
                     │            │                                          │                                                                                                                                                                                     │
(0, 6): ────Y^-0.5───@───Y^0.5────@──────────────Y^-0.5──────────────────────@─────────Y^0.5────────────────────────────────────────────@────────────────Y^-0.5───────────────@──────────Y^0.5─────@───────────────────────